In [2]:
#r "nuget: StackExchange.Redis"


Installed Packages StackExchange.Redis, 2.7.10

In [3]:
/* sample server code */

using StackExchange.Redis;
using System;
using System.Threading;

var options = ConfigurationOptions.Parse("78.109.201.86:6379"); // host1:port1, host2:port2, ...
options.Password = "1qazxsw2$$";

string redisConnectionString = "localhost:6379";
// string channelName = "sampleChannel";
string channelName = "f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch";
string streamKey = "sampleStream";

ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(options);
// ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(redisConnectionString);
ISubscriber subscriber = redis.GetSubscriber();

subscriber.Subscribe(channelName, (channel, message) =>
{
    Console.WriteLine($"Received message from channel '{channel}': {message}");
});


var st = DateTime.Now;


while(DateTime.Now - st < TimeSpan.FromSeconds(10)){
    Thread.Sleep(1000);
}

redis.Dispose();


Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test
Received message from channel 'f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch': test


In [1]:
using StackExchange.Redis;
using System;

var options = ConfigurationOptions.Parse("78.109.201.86:6379"); // host1:port1, host2:port2, ...
options.Password = "1qazxsw2$$";

string redisConnectionString = "localhost:6379";
// string channelName = "sampleChannel";
string channelName = "f7e58e26-f9c0-44e7-8954-08dc093dd85d.ch";
string streamKey = "sampleStream";

ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(options);
// ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(redisConnectionString);
ISubscriber subscriber = redis.GetSubscriber();

subscriber.Subscribe(channelName, (channel, message) =>
{
    Console.WriteLine($"Received message from channel '{channel}': {message}");
});

await subscriber.PublishAsync(channelName, "Hello from Server 1!");

redis.Dispose();


Error: (1,7): error CS0246: The type or namespace name 'StackExchange' could not be found (are you missing a using directive or an assembly reference?)
(4,15): error CS0103: The name 'ConfigurationOptions' does not exist in the current context
(12,1): error CS0246: The type or namespace name 'ConnectionMultiplexer' could not be found (are you missing a using directive or an assembly reference?)
(14,1): error CS0246: The type or namespace name 'ISubscriber' could not be found (are you missing a using directive or an assembly reference?)
(12,37): error CS0103: The name 'ConnectionMultiplexer' does not exist in the current context

In [46]:
using StackExchange.Redis;
using System;

var options = ConfigurationOptions.Parse("78.109.201.86:6379"); // host1:port1, host2:port2, ...
options.Password = "1qazxsw2$$";

// string redisConnectionString = "localhost:6379";
string channelName = "sampleChannel";
string streamKey = "sampleStream";
string consumerGroupName = "sampleGroup";

// ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(redisConnectionString);
ConnectionMultiplexer redis = await ConnectionMultiplexer.ConnectAsync(options);

IDatabase redisDb = redis.GetDatabase();

for(var i = 0 ; i < 10 ; i++){
var message = new NameValueEntry("messageField", "Message from Server 1");

var messageId = await redisDb.StreamAddAsync(streamKey, new [] {message});
Console.WriteLine($"Added message to stream '{streamKey}' with ID: {messageId}");

}


// Create a consumer group associated with the stream
try{
    redisDb.StreamCreateConsumerGroup(streamKey, consumerGroupName, StreamPosition.NewMessages);
}catch (Exception e){
    Console.WriteLine($"consumer group already exist 'I THINK' : {e.Message}");
}

// this is not accurate (always returning 4)
// RedisResult result = await redisDb.ExecuteAsync("XPENDING", streamKey, consumerGroupName);
// Console.WriteLine($"Total pending messages: {result}");

// var streamMessages = await redisDb.StreamReadAsync(streamKey , "0");
// foreach (var message in streamMessages)
// {
//     Console.WriteLine($"Received message from stream '{streamKey}': ID:{message.Id} VAL0:{message.Values[0]}");
//     var t = await redisDb.StreamAcknowledgeAsync(streamKey , consumerGroupName, message.Id);
// }
// // Delete acknowledged messages from the stream
// await redisDb.StreamDeleteAsync(streamKey, streamMessages.Select(msg => msg.Id).ToArray());

redis.Dispose();


Added message to stream 'sampleStream' with ID: 1704116257792-0
Added message to stream 'sampleStream' with ID: 1704116257897-0
Added message to stream 'sampleStream' with ID: 1704116257964-0
Added message to stream 'sampleStream' with ID: 1704116258033-0
Added message to stream 'sampleStream' with ID: 1704116258116-0
Added message to stream 'sampleStream' with ID: 1704116258186-0
Added message to stream 'sampleStream' with ID: 1704116258238-0
Added message to stream 'sampleStream' with ID: 1704116258303-0
Added message to stream 'sampleStream' with ID: 1704116258358-0
Added message to stream 'sampleStream' with ID: 1704116258407-0
consumer group already exist 'I THINK' : BUSYGROUP Consumer Group name already exists
